In [40]:
import os
import sys
import numpy as np
from scipy.spatial import distance
from numba import jit
from tqdm import tqdm_notebook

In [342]:
# def get_des(dir_name, descriptor_type ,parent_path=''):

In [54]:
def grid_based_charge_train(dir_name, sigma, parent_path=''):
    vec, coor_list, chg, ngxf, ngyf, ngzf = split_read_chgcar(
        dir_name, parent_path)

    chg_den_flat = charge_label(vec, chg)  # charge data manipulation
    charge_arr = chg_den_flat.reshape((-1, 1))

    coor_cart_list = dir2cart(vec, coor_list)

    sigma_size = sigma.size
    des_arr_all = np.zeros(
        (ngxf * ngyf * ngzf, 5 * sigma_size * len(coor_list)))
    grid_coor = grid_gen_nb(ngxf, ngyf, ngzf, vec)

    for i, coor_cart in tqdm_notebook(enumerate(coor_cart_list)):

        start_index = 5 * sigma_size * i  # starting index of each element
        end_index = 5 * sigma_size * (i + 1)  # ending index of each element

        dist_arr_stage_0 = comp_gen(grid_coor, coor_cart)
        dist_arr_stage_1 = dist_gen(dist_arr_stage_0, grid_coor, coor_cart)
        dist_arr_list = batch_gen(dist_arr_stage_1, 50000)

        pointer_grid = 0
        for dist_arr in tqdm_notebook(dist_arr_list):
            
            step_forward = dist_arr.shape[0]
            
            s_ini, v_ini, t_ini = des_rudi_gen(dist_arr, sigma)
            des_arr = invar_gen(s_ini, v_ini, t_ini)

            des_arr_all[pointer_grid:(pointer_grid + step_forward),
                        start_index:end_index] = des_arr

            pointer_grid += step_forward
    
    os.chdir('/'.join((parent_path, dir_name)))
    np.save("x.npy", des_arr_all)
    np.save("y_charge.npy", charge_arr)    
    os.chdir(sys.path[0])
    

In [54]:
def grid_based_charge_predict(dir_name, sigma, parent_path=''):
    vec, coor_list, is_cart = read_poscar(dir_name, parent_path='')
    encut, prec = read_incar(['ENCUT','PREC'], dir_name, parent_path)
    
    ngxf, ngyf, ngzf = ngf_regen(encut, prec)
    
#     chg_den_flat = charge_label(vec, chg)  # charge data manipulation
#     charge_arr = chg_den_flat.reshape((-1, 1))
    if not is_cart:
        coor_cart_list = dir2cart(vec, coor_list)
    else:
        coor_cart_list = coor_list
    
    sigma_size = sigma.size
    des_arr_all = np.zeros(
        (ngxf * ngyf * ngzf, 5 * sigma_size * len(coor_list)))
    grid_coor = grid_gen_nb(ngxf, ngyf, ngzf, vec)

    for i, coor_cart in tqdm_notebook(enumerate(coor_cart_list)):

        start_index = 5 * sigma_size * i  # starting index of each element
        end_index = 5 * sigma_size * (i + 1)  # ending index of each element

        dist_arr_stage_0 = comp_gen(grid_coor, coor_cart)
        dist_arr_stage_1 = dist_gen(dist_arr_stage_0, grid_coor, coor_cart)
        dist_arr_list = batch_gen(dist_arr_stage_1, 50000)

        pointer_grid = 0
        for dist_arr in tqdm_notebook(dist_arr_list):
            
            step_forward = dist_arr.shape[0]
            
            s_ini, v_ini, t_ini = des_rudi_gen(dist_arr, sigma)
            des_arr = invar_gen(s_ini, v_ini, t_ini)

            des_arr_all[pointer_grid:(pointer_grid + step_forward),
                        start_index:end_index] = des_arr

            pointer_grid += step_forward
    
    os.chdir('/'.join((parent_path, dir_name)))
    np.save("x.npy", des_arr_all)
    np.save("y_charge.npy", charge_arr)    
    os.chdir(sys.path[0])
    

In [55]:
# sigma = np.exp(np.linspace(np.log(0.25), np.log(5), 16))

# for i in tqdm_notebook(range(3775,3795)):
#     grid_based_charge_train(str(i),sigma \
#                   ,'/home/caizefeng/ml_dft/ML_DFT_03272019/data/chgcar')

1601.92s/it

In [41]:
def split_read_chgcar(dir_name, parent_path=''):
    chgcar_name = '/'.join((parent_path, dir_name, 'CHGCAR'))
    with open(chgcar_name, 'r') as f:
        f_vec = open(chgcar_name + '_vec', 'w')
        f_chg = open(chgcar_name + '_chg', 'w')
        for line_num, line in enumerate(f):
            if line_num < 8:
                if line_num == 1:
                    scaling = float(line.split()[0])
                elif (line_num >= 2) and (line_num <= 4):
                    f_vec.write(line.rstrip() + '\n')
                elif line_num == 5:
                    atom_list = line.split()
                    atom_type_num = len(atom_list)
                elif line_num == 6:
                    atom_num_list = [int(i) for i in line.split()]
                    atom_num = sum(atom_num_list)
                    phantom_list = list(atom_num_list)  # copy the list
                    phantom_list.insert(0, 0)
                    # for cumulative summation after
                    phantom_list_np = np.cumsum(np.array(phantom_list))
                    f_coor = list(range(atom_type_num))
                    for index in range(atom_type_num):
                        f_coor[index] = open(chgcar_name + '_coor_' +
                                             str(index), 'w')   
            else:
                if line_num < 10 + atom_num:
                    if line_num <= (7 + atom_num):
                        j = 0
                        while j <= atom_type_num - 1:
                            if (8 + phantom_list_np[j]) <= line_num <= (7 + phantom_list_np[j+1]):
                                f_coor[j].write(line.rstrip() + '\n')
                                break
                            j += 1
                    elif line_num == (9 + atom_num):
                        ngxf, ngyf, ngzf = [int(i) for i in line.split()]
                        ngf = ngxf * ngyf * ngzf
                        chg_line_num = ngf // 5
                        chg_space = 5 - ngf % 5
                else:
                    if (line_num >= (10 + atom_num)) and (line_num <= (9 + atom_num + chg_line_num)):
                        f_chg.write(line.rstrip() + '\n')
                    elif chg_space != 5:
                        # lines is multiples of 5 or not determines whether patching is needed
                        if line_num == (10 + atom_num + chg_line_num):
                            f_chg.write(line.rstrip() +
                                        chg_space * ' 0' + '\n')
        f_vec.close()
        for i in range(atom_type_num):
            f_coor[i].close()
        f_chg.close()

    vec = scaling * np.loadtxt(chgcar_name + '_vec', dtype=np.float64)
    coor_list = []
    for index in range(atom_type_num):
        coor_list.append(np.loadtxt(chgcar_name + '_coor_' +
                                    str(index), dtype=np.float64))
    chg = np.loadtxt(chgcar_name + '_chg', dtype=np.float64)

    return vec, coor_list, chg, ngxf, ngyf, ngzf

In [41]:
def read_poscar(dir_name, parent_path=''):
    poscar_name = '/'.join((parent_path, dir_name, 'POSCAR'))
    with open(poscar_name, 'r') as f:
        f_vec = open(poscar_name + '_vec', 'w')
#         f_chg = open(chgcar_name + '_chg', 'w')
        for line_num, line in enumerate(f):
            if line_num < 8:
                if line_num == 1:
                    scaling = float(line.split()[0])
                elif (line_num >= 2) and (line_num <= 4):
                    f_vec.write(line.rstrip() + '\n')
                elif line_num == 5:
                    atom_list = line.split()
                    atom_type_num = len(atom_list)
                elif line_num == 6:
                    atom_num_list = [int(i) for i in line.split()]
                    atom_num = sum(atom_num_list)
                    phantom_list = list(atom_num_list)  # copy the list
                    phantom_list.insert(0, 0)
                    # for cumulative summation after
                    phantom_list_np = np.cumsum(np.array(phantom_list))
                    f_coor = list(range(atom_type_num))
                    for index in range(atom_type_num):
                        f_coor[index] = open(chgcar_name + '_coor_' +
                                             str(index), 'w')
                elif line_num == 7:
                    is_cart = list(line.split()[0])[0] == 'C'
            
            elif line_num < 10 + atom_num:
                if line_num <= (7 + atom_num):
                    j = 0
                    while j <= atom_type_num - 1:
                        if (8 + phantom_list_np[j]) <= line_num <= (7 + phantom_list_np[j+1]):
                            f_coor[j].write(line.rstrip() + '\n')
                            break
                        j += 1

#                     elif line_num == (9 + atom_num):
#                         ngxf, ngyf, ngzf = [int(i) for i in line.split()]
#                         ngf = ngxf * ngyf * ngzf
#                         chg_line_num = ngf // 5
#                         chg_space = 5 - ngf % 5
#                 else:
#                     if (line_num >= (10 + atom_num)) and (line_num <= (9 + atom_num + chg_line_num)):
#                         f_chg.write(line.rstrip() + '\n')
#                     elif chg_space != 5:
#                         # lines is multiples of 5 or not determines whether patching is needed
#                         if line_num == (10 + atom_num + chg_line_num):
#                             f_chg.write(line.rstrip() +
#                                         chg_space * ' 0' + '\n')
        f_vec.close()
        for i in range(atom_type_num):
            f_coor[i].close()
#         f_chg.close()

    vec = scaling * np.loadtxt(chgcar_name + '_vec', dtype=np.float64)
    coor_list = []
    for index in range(atom_type_num):
        coor_list.append(np.loadtxt(chgcar_name + '_coor_' +
                                    str(index), dtype=np.float64))

    return vec, coor_list, is_cart

In [ ]:
def read_incar(tag,dir_name, parent_path=''):
    incar_name = '/'.join((parent_path, dir_name, 'INCAR'))

In [46]:
def dir2cart(vec, coor_list):
    return [coor @ vec for coor in coor_list]

In [47]:
@jit(["float64[:,:](uint16, uint16, uint16, float64[:,:])"], nopython=True)
def grid_gen_nb(ngxf, ngyf, ngzf, vec):
    grid_coor = np.zeros((ngxf*ngyf*ngzf, 3))
    index = 0
    for z in np.linspace(0, 1, ngzf + 1):
        if z == 1:  # jit can optimize "endpoint" argument. manually instead
            continue
        for y in np.linspace(0, 1, ngyf + 1):
            if y == 1:
                continue
            for x in np.linspace(0, 1, ngxf + 1):
                if x == 1:
                    continue
                grid_coor[index] = np.array([x, y, z]) @ vec
                index += 1
    return grid_coor

In [48]:
@jit(["float64[:,:,:](float64[:,:], float64[:,:])"], nopython=True)
def comp_gen(grid_coor, coor_cart):  # components of xyz direction

    dist_arr = np.zeros((grid_coor.shape[0], coor_cart.shape[0], 4))
    for axis in range(3):
        for grid in range(grid_coor.shape[0]):
            dist_arr[grid, :, axis + 1] = coor_cart[:, axis] - \
                grid_coor[grid, axis]

    return dist_arr

In [49]:
@jit(["float64[:,:,:](float64[:,:,:], float64[:,:], float64[:,:])"])
def dist_gen(dist_arr, grid_coor, coor_cart):
    dist_arr[:, :, 0] = distance.cdist(grid_coor, coor_cart, 'euclidean')
    return dist_arr

In [50]:
def batch_gen(dist_arr, batch_size):
    batch_num = dist_arr.shape[0] // batch_size
    sep_list = [batch_size * (i+1) for i in range(batch_num)]
    return np.split(dist_arr, sep_list)

In [51]:
@jit(["int64[:,:](float64[:,:], float64)"], nopython=True)
def cutoff_gen(array_in, r_cutoff):
    return np.where(array_in > r_cutoff, 0, 1)

In [52]:
@jit('Tuple((float64[:,:,:], float64[:,:,:], float64[:,:,:,:]))(float64[:,:,:], float64[:])', nopython=True, parallel=True)
def des_rudi_gen(dist_arr, sigma):  # generate feature without invariance
    sigma_size = sigma.size
    s_ini = np.zeros((dist_arr.shape[0], 1, sigma_size))
    v_ini = np.zeros((dist_arr.shape[0], 3, sigma_size))
    t_ini = np.zeros((dist_arr.shape[0], 3, 3, sigma_size))
    f_cutoff = cutoff_gen(dist_arr[:, :, 0], 9)
    gauss_ini = np.exp(- (dist_arr[:, :, 0]**2) / 2) * f_cutoff

    for index, sig in enumerate(sigma):
        c_k = 1/((2*np.pi)**1.5 * sig**3)
        gauss = gauss_ini ** (1/sig ** 2)
        vari = 2 * sig**2
        vari_square = vari ** 2

        s_ini[:, 0, index] = c_k * \
            np.sum(gauss,
                   axis=1)
        for i in range(3):
            v_ini[:, i, index] = c_k * \
                np.sum(dist_arr[:, :, i + 1]/vari * gauss,
                       axis=1)

        for i in range(3):
            for j in range(3):
                t_ini[:, i, j, index] = c_k * \
                    np.sum(dist_arr[:, :, i + 1] * dist_arr[:, :, j + 1]/vari_square * gauss,
                           axis=1)
    return s_ini, v_ini, t_ini

In [53]:
@jit(['float64[:,:](float64[:,:,:],float64[:,:,:],float64[:,:,:,:])'], nopython=True, parallel=True)
def invar_gen(s_ini, v_ini, t_ini):  # process feature for rotational invariance
    des_arr = np.zeros((s_ini.shape[0], 5 * s_ini.shape[2]))
    sigma_size = s_ini.shape[2]
    for index in range(sigma_size):

        des_arr[:, index] = s_ini[:, 0, index]

        des_arr[:, index + sigma_size] = np.sqrt(np.sum(
            v_ini[:, :, index] ** 2, axis=1))

# printed wrong version
#         des_arr[:, index + 2 * sigma_size] = t_ini[:, 0, 0, index] ** 2 + \
#             t_ini[:, 1, 1, index] ** 2 + t_ini[:, 2, 2, index] ** 2

        des_arr[:, index + 2 * sigma_size] = t_ini[:, 0, 0, index] + \
            t_ini[:, 1, 1, index] + t_ini[:, 2, 2, index]

        des_arr[:, index + 3 * sigma_size] = t_ini[:, 0, 0, index] * \
            t_ini[:, 1, 1, index] + t_ini[:, 1, 1, index] * t_ini[:, 2, 2, index] \
            + t_ini[:, 0, 0, index] * t_ini[:, 2, 2, index] - \
            t_ini[:, 0, 1, index] ** 2 - t_ini[:, 1, 2, index] ** 2 \
            - t_ini[:, 0, 2, index] ** 2

        des_arr[:, index + 4 * sigma_size] = \
            t_ini[:, 0, 0, index] * (t_ini[:, 1, 1, index]*t_ini[:, 2, 2, index]-t_ini[:, 1, 2, index]*t_ini[:, 2, 1, index]) \
            - t_ini[:, 0, 1, index] * (t_ini[:, 1, 0, index]*t_ini[:, 2, 2, index]-t_ini[:, 1, 2, index]*t_ini[:, 2, 0, index]) \
            + t_ini[:, 0, 2, index]*(t_ini[:, 1, 0, index]*t_ini[:, 2,
                                                                 1, index]-t_ini[:, 1, 1, index]*t_ini[:, 2, 0, index])

    return des_arr